# Heat Transmission Calculation Notebook

This notebook sets up a Python environment and demonstrates interactive forms and dropdowns for heat transmission calculations using Jupyter widgets.

## Install and Import Jupyter Widgets

The following cell ensures `ipywidgets` is installed and imports the necessary modules for interactive widgets.

In [1]:
# Install ipywidgets if not already installed (uncomment if needed)
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

## Warmtedoorgangscoëfficiënt (U-waarde)

$$U = \frac{1}{R_e + R_c + R_i} \quad \left[\frac{W}{m^2 \cdot K}\right]$$

| Symbool | Omschrijving | Standaardwaarde |
|---------|-------------|-----------------|
| $R_i$ | Warmte-overgangsweerstand binnenzijde | 0,13 m²·K/W |
| $R_e$ | Warmte-overgangsweerstand buitenzijde | 0,04 m²·K/W |
| $R_c$ | Warmteweerstand constructie | $\sum d / \lambda$ |

Voeg lagen toe via de knop hieronder. Per laag kies je een materiaal uit de lijst of vul je de R-waarde handmatig in (voor bijv. luchtspouw).


In [2]:
import json
import importlib
import ipywidgets as widgets
from IPython.display import display, HTML

import heat_calc
importlib.reload(heat_calc)                          # pick up any edits without restarting kernel
from heat_calc import SURFACE_R, LayerWidget

# ── Load material data ────────────────────────────────────────────────────────
with open('material_properties.json', 'r', encoding='utf-8') as f:
    materials = json.load(f)

# ── UI state ──────────────────────────────────────────────────────────────────
layers     = []
layers_box = widgets.VBox([])
out        = widgets.Output()

ri_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Binnenzijde  —  Ri = 0,13 m²·K/W',
    description='Ri (binnen):',
    layout=widgets.Layout(width='340px')
)
re_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Buitenzijde  —  Re = 0,04 m²·K/W',
    description='Re (buiten):',
    layout=widgets.Layout(width='340px')
)
ri_dd.observe(lambda _: refresh(), names='value')
re_dd.observe(lambda _: refresh(), names='value')

add_btn = widgets.Button(
    description='＋ Voeg laag toe', button_style='primary',
    layout=widgets.Layout(width='160px')
)


def refresh(*_):
    """Recompute totals and render the result table."""
    ri = SURFACE_R[ri_dd.value]
    re = SURFACE_R[re_dd.value]

    with out:
        out.clear_output(wait=True)

        rows_html = ''
        total_d   = 0.0
        total_rc  = 0.0

        rows_html += (
            f'<tr class="surface"><td>lucht (binnen)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{ri:.2f}</td></tr>\n'
        )

        for layer in layers:
            info  = layer.row_info()
            r     = info['R']
            d     = info['d']
            d_str = f'{d:.3f}' if isinstance(d, float) else '—'
            r_str = f'{r:.3f}' if r is not None else '?'
            if isinstance(d, float): total_d  += d
            if r is not None:        total_rc += r
            rows_html += (
                f'<tr><td>{info["naam"]}</td>'
                f'<td>{d_str}</td><td>{info["lam"]}</td>'
                f'<td>{r_str}</td><td>—</td></tr>\n'
            )

        rows_html += (
            f'<tr class="surface"><td>lucht (buiten)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{re:.2f}</td></tr>\n'
        )

        total_r = ri + total_rc + re
        u       = 1.0 / total_r if total_r > 0 else None
        u_str   = f'{u:.3f}' if u is not None else '?'
        d_tot   = f'{total_d:.3f}' if total_d > 0 else '—'

        html = f"""
<style>
  .utbl {{ border-collapse:collapse; font-family:sans-serif;
           font-size:13px; min-width:680px; }}
  .utbl th {{ background:#2c5f8a; color:white;
              padding:6px 12px; text-align:left; }}
  .utbl td {{ padding:5px 12px; border-bottom:1px solid #ddd; }}
  .utbl tr:not(.total-row):not(.u-row):hover td {{ background:#f0f7ff; }}
  .utbl .surface td  {{ color:#555; font-style:italic; }}
  .utbl .total-row td {{ font-weight:bold; background:#e8f0fe;
                         border-top:2px solid #2c5f8a; }}
  .utbl .u-row td   {{ font-weight:bold; background:#2c5f8a;
                       color:white; font-size:15px; }}
</style>
<table class="utbl">
  <tr>
    <th>Materiaal / Laag</th>
    <th>d [m]</th>
    <th>λ [W/(m·K)]</th>
    <th>R = d/λ [m²·K/W]</th>
    <th>Ri &amp; Re [m²·K/W]</th>
  </tr>
  {rows_html}
  <tr class="total-row">
    <td>TOTAAL</td>
    <td>{d_tot}</td>
    <td>—</td>
    <td>{total_rc:.3f}</td>
    <td>{ri + re:.2f}</td>
  </tr>
  <tr class="u-row">
    <td colspan="5">
      U = 1 / (R<sub>i</sub> {ri:.2f} + R<sub>c</sub> {total_rc:.3f}
              + R<sub>e</sub> {re:.2f})
      &nbsp;=&nbsp; <b>{u_str} W/(m²·K)</b>
    </td>
  </tr>
</table>"""
        display(HTML(html))


def add_layer(_=None):
    layer = LayerWidget(materials, update_cb=refresh, remove_cb=remove_layer)
    layers.append(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


def remove_layer(layer):
    layers.remove(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


add_btn.on_click(add_layer)

display(widgets.VBox([
    widgets.HTML('<b>Overgangsweerstanden</b>'),
    widgets.HBox([ri_dd, re_dd]),
    widgets.HTML('<b style="margin-top:10px;display:block">Constructielagen (Rc)</b>'),
    layers_box,
    add_btn,
    widgets.HTML('<b style="margin-top:10px;display:block">Resultaat</b>'),
    out,
]))

# Start with one empty layer
add_layer()


## Fk